#### Case 3b: Liquid hydrogen ($LH_2$) in a 2033 $\text{m}^3$ tank

This application corresponds to NASA Multi-purpose Hydrogen Test Bed. It consists of a 18.09 $m^3$ storage tank [NASA](Spray bar zero-gravity vent system for on-orbit liquid hydrogen storage(2003)).

* q = 1.413 $W m^{-2}$
* Non-isobaric storage but isobaric solution allows an estimation of the evaporative fraction.

For purposes of the example, it is assumed that the storage tank is passively insulated with perlite.

In [ ]:
# Ensure that python finds the submodules
import sys
sys.path.append("..") # Adds higher directory to python modules path.

# Scientific computing
import numpy as np

# Visualisation
import matplotlib.pyplot as plt

# Import the storage tank Class
from cryoevap.storage_tanks import Tank

# Import Cryogen class
from cryoevap.cryogens import Cryogen

#### Setup tank and cryogen properties


In [ ]:
# Cylindrical storage tank properties
Q_roof = 0 # Roof heat ingress / W
d_i = 3.05 # Internal diameter / m
d_o = 3.25   # External diameter / m
T_air = 293.15 # Temperature of the environment K

# Set overall heat transfer coefficient through the walls for liquid and vapour
U_L = 5.187e-3 # W/m2/K
U_V = 5.187e-3 # W/m2/K

# Specify heat transfer rate at the bottom
# This will represent the heat conduction from the piping system
Q_b = 0 # W, 

# Vertically orientated cylindrical tank volume
V_tank = 18.09 #m^3

# Initial liquid filling / Dimensionless
LF = 0.90 

# Specify tank operating pressure
P = 113000 # Pa

# Initialize mid-scale tank
mid_tank = Tank(d_i, d_o, V_tank, LF)
mid_tank.set_HeatTransProps(U_L, U_V, T_air, Q_roof, Q_b, eta_w=0.876)

# Keep the tank roof insulated
mid_tank.U_roof = U_V

# Initialise cryogen
hydrogen = Cryogen(name = "hydrogen")
hydrogen.set_coolprops(P)

# Set cryogen
mid_tank.cryogen = hydrogen

In [ ]:
1.413/(293.15-20.75)

Calculate initial evaporation rate and transient period

In [ ]:
# Calculate initial evaporation rate
print("The initial evaporation rate of " + hydrogen.name + " is %.1f kg/h" % (mid_tank.b_l_dot * 3600))

# Estimate transient period duration
print("Transient period = %.3f s " % mid_tank.tau)

# Minimum number of hours to achieve steady state 
tau_h = (np.floor(mid_tank.tau / 3600) + 1)

# Print simulation time of the transient period for short-term storage
print("Simulation time: %.0i h" % tau_h )

# Calculate boil-off rate
BOR = (mid_tank.b_l_dot * 24 * 3600) / (mid_tank.V * mid_tank.LF * mid_tank.cryogen.rho_L)
print("BOR = %.3f %%" % (BOR * 100))

### Simulation setup and execution

In [ ]:
# Define vertical spacing
dz = 0.01

# Calculate number of nodes
n_z = 1 + int(np.round(mid_tank.l_V/dz, 0))

# Define dimensionless computational grid
mid_tank.z_grid = np.linspace(0, 1, n_z)

# Define evaporation time as twice as the transient period
evap_time = 3600 *tau_h * 2

# Time step to plot each vapour temperature profile
mid_tank.plot_interval = evap_time/6

# Time step to record data, relevant for plotting integrated quantities such as
# the vapour to liquid heat transfer rate, Q_VL
mid_tank.time_interval = 60

mid_tank.evaporate(evap_time)

### Visualisation of results

#### Vapour temperature

In [ ]:
mid_tank.plot_tv()

ax = plt.gca()
ax.hlines(y=0.9, xmin=21, xmax = 26)

In [ ]:
mid_tank.sol.y

Visualise liquid and vapour heat ingresses, $\dot{Q}_{\text{L}}$ and  $\dot{Q}_{\text{V}}$.

The plot also shows the vapour to liquid heat ingress, $\dot{Q}_{VL}$, and  the partition of the vapour heat ingress that is transferred to the interface by the wall directly, $\dot{Q}_{\text{V,w}}$

In [ ]:
mid_tank.plot_Q()

#### Plot liquid volume

In [ ]:
mid_tank.plot_V_L()

#### Plot evaporation rate, $\dot{B}_{\text{L}}$, and boil-off gas rate, $\dot{B}_{}$

In [ ]:
mid_tank.plot_BOG(unit='kg/h')

Optional: CSV data export

If evaporation data is intended to be post-processed in another software, it can be exported readily with the help of the Pandas package.

In [ ]:
# Import pandas 
import pandas as pd

In [ ]:
# Create dataframe from dictionary
df_evap = pd.DataFrame.from_dict(mid_tank.data)

# Save file to the current working directory
df_evap.to_csv('iso_LH2_2033m3.csv')

# Show the first five columns of the dataframe in console
df_evap.head()

#### References

U.S. Department of Energy. (2021, October). DOE/NASA Advances in Liquid Hydrogen Storage Workshop. Retrieved from [https://www.energy.gov/sites/default/files/2021-10/new-lh2-sphere.pdf]